# Starbucks Capstone Challenge

### Introduction

This data set contains simulated data that mimics customer behavior on the Starbucks rewards mobile app. Once every few days, Starbucks sends out an offer to users of the mobile app. An offer can be merely an advertisement for a drink or an actual offer such as a discount or BOGO (buy one get one free). Some users might not receive any offer during certain weeks. 

Not all users receive the same offer, and that is the challenge to solve with this data set.

Your task is to combine transaction, demographic and offer data to determine which demographic groups respond best to which offer type. This data set is a simplified version of the real Starbucks app because the underlying simulator only has one product whereas Starbucks actually sells dozens of products.

Every offer has a validity period before the offer expires. As an example, a BOGO offer might be valid for only 5 days. You'll see in the data set that informational offers have a validity period even though these ads are merely providing information about a product; for example, if an informational offer has 7 days of validity, you can assume the customer is feeling the influence of the offer for 7 days after receiving the advertisement.

You'll be given transactional data showing user purchases made on the app including the timestamp of purchase and the amount of money spent on a purchase. This transactional data also has a record for each offer that a user receives as well as a record for when a user actually views the offer. There are also records for when a user completes an offer. 

Keep in mind as well that someone using the app might make a purchase through the app without having received an offer or seen an offer.

### Example

To give an example, a user could receive a discount offer buy 10 dollars get 2 off on Monday. The offer is valid for 10 days from receipt. If the customer accumulates at least 10 dollars in purchases during the validity period, the customer completes the offer.

However, there are a few things to watch out for in this data set. Customers do not opt into the offers that they receive; in other words, a user can receive an offer, never actually view the offer, and still complete the offer. For example, a user might receive the "buy 10 dollars get 2 dollars off offer", but the user never opens the offer during the 10 day validity period. The customer spends 15 dollars during those ten days. There will be an offer completion record in the data set; however, the customer was not influenced by the offer because the customer never viewed the offer.

### Cleaning

This makes data cleaning especially important and tricky.

You'll also want to take into account that some demographic groups will make purchases even if they don't receive an offer. From a business perspective, if a customer is going to make a 10 dollar purchase without an offer anyway, you wouldn't want to send a buy 10 dollars get 2 dollars off offer. You'll want to try to assess what a certain demographic group will buy when not receiving any offers.

### Final Advice

Because this is a capstone project, you are free to analyze the data any way you see fit. For example, you could build a machine learning model that predicts how much someone will spend based on demographics and offer type. Or you could build a model that predicts whether or not someone will respond to an offer. Or, you don't need to build a machine learning model at all. You could develop a set of heuristics that determine what offer you should send to each customer (i.e., 75 percent of women customers who were 35 years old responded to offer A vs 40 percent from the same demographic to offer B, so send offer A).

# Data Sets

The data is contained in three files:

* portfolio.json - containing offer ids and meta data about each offer (duration, type, etc.)
* profile.json - demographic data for each customer
* transcript.json - records for transactions, offers received, offers viewed, and offers completed

Here is the schema and explanation of each variable in the files:

**portfolio.json**
* id (string) - offer id
* offer_type (string) - type of offer ie BOGO, discount, informational
* difficulty (int) - minimum required spend to complete an offer
* reward (int) - reward given for completing an offer
* duration (int) - time for offer to be open, in days
* channels (list of strings)

**profile.json**
* age (int) - age of the customer 
* became_member_on (int) - date when customer created an app account
* gender (str) - gender of the customer (note some entries contain 'O' for other rather than M or F)
* id (str) - customer id
* income (float) - customer's income

**transcript.json**
* event (str) - record description (ie transaction, offer received, offer viewed, etc.)
* person (str) - customer id
* time (int) - time in hours since start of test. The data begins at time t=0
* value - (dict of strings) - either an offer id or transaction amount depending on the record

**Note:** If you are using the workspace, you will need to go to the terminal and run the command `conda update pandas` before reading in the files. This is because the version of pandas in the workspace cannot read in the transcript.json file correctly, but the newest version of pandas can. You can access the termnal from the orange icon in the top left of this notebook.  

You can see how to access the terminal and how the install works using the two images below.  First you need to access the terminal:

<img src="pic1.png"/>

Then you will want to run the above command:

<img src="pic2.png"/>

Finally, when you enter back into the notebook (use the jupyter icon again), you should be able to run the below cell without any errors.

In [197]:
import pandas as pd
import numpy as np
import math
import json
%matplotlib inline

# read in the json files
portfolio = pd.read_json('data/portfolio.json', orient='records', lines=True)
profile = pd.read_json('data/profile.json', orient='records', lines=True)
transcript = pd.read_json('data/transcript.json', orient='records', lines=True)

In [198]:
profile.head(10)

,gender,age,id,became_member_on,income
0,None,118,68be06ca386d4c31939f3a4f0e3dd783,20170212,NaN
1,F,55,0610b486422d4921ae7d2bf64640c50b,20170715,112000.0
2,None,118,38fe809add3b4fcf9315a9694bb96ff5,20180712,NaN
3,F,75,78afa995795e4d85b5d9ceeca43f5fef,20170509,100000.0
4,None,118,a03223e636434f42ac4c3df47e8bac43,20170804,NaN
5,M,68,e2127556f4f64592b11af22de27a7932,20180426,70000.0
6,None,118,8ec6ce2a7e7949b1bf142def7d0e0586,20170925,NaN
7,None,118,68617ca6246f4fbc85e91a2a49552598,20171002,NaN
8,M,65,389bc3fa690240e798340f5a15918d5c,20180209,53000.0
9,None,118,8974fc5686fe429db53ddde067b88302,20161122,NaN


In [199]:
profile.isnull().sum()

gender              2175
age                    0
id                     0
became_member_on       0
income              2175
dtype: int64

In [200]:
profile.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17000 entries, 0 to 16999
Data columns (total 5 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   gender            14825 non-null  object 
 1   age               17000 non-null  int64  
 2   id                17000 non-null  object 
 3   became_member_on  17000 non-null  int64  
 4   income            14825 non-null  float64
dtypes: float64(1), int64(2), object(2)
memory usage: 664.2+ KB


In [201]:
transcript.head(30)

,person,event,value,time
0,78afa995795e4d85b5d9ceeca43f5fef,offer received,{'offer id': '9b98b8c7a33c4b65b9aebfe6a799e6d9'},0
1,a03223e636434f42ac4c3df47e8bac43,offer received,{'offer id': '0b1e1539f2cc45b7b9fa7c272da2e1d7'},0
2,e2127556f4f64592b11af22de27a7932,offer received,{'offer id': '2906b810c7d4411798c6938adc9daaa5'},0
3,8ec6ce2a7e7949b1bf142def7d0e0586,offer received,{'offer id': 'fafdcd668e3743c1bb461111dcafc2a4'},0
4,68617ca6246f4fbc85e91a2a49552598,offer received,{'offer id': '4d5c57ea9a6940dd891ad53e9dbe8da0'},0
5,389bc3fa690240e798340f5a15918d5c,offer received,{'offer id': 'f19421c1d4aa40978ebb69ca19b0e20d'},0
6,c4863c7985cf408faee930f111475da3,offer received,{'offer id': '2298d6c36e964ae4a3e7e9706d1fb8c2'},0
7,2eeac8d8feae4a8cad5a6af0499a211d,offer received,{'offer id': '3f207df678b143eea3cee63160fa8bed'},0
8,aa4862eba776480b8bb9c68455b8c2e1,offer received,{'offer id': '0b1e1539f2cc45b7b9fa7c272da2e1d7'},0
9,31dda685af34476cad5bc968bdb01c53,offer received,{'offer id': '0b1e1539f2cc45b7b9fa7c272da2e1d7'},0


In [202]:
transcript.isnull().sum()

person    0
event     0
value     0
time      0
dtype: int64

In [203]:
transcript.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 306534 entries, 0 to 306533
Data columns (total 4 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   person  306534 non-null  object
 1   event   306534 non-null  object
 2   value   306534 non-null  object
 3   time    306534 non-null  int64 
dtypes: int64(1), object(3)
memory usage: 9.4+ MB


In [204]:
len(transcript)

306534

In [205]:
len(profile)

17000

In [206]:
len(portfolio)

10

In [207]:
portfolio.head(10)

,reward,channels,difficulty,duration,offer_type,id
0,10,"[email, mobile, social]",10,7,bogo,ae264e3637204a6fb9bb56bc8210ddfd
1,10,"[web, email, mobile, social]",10,5,bogo,4d5c57ea9a6940dd891ad53e9dbe8da0
2,0,"[web, email, mobile]",0,4,informational,3f207df678b143eea3cee63160fa8bed
3,5,"[web, email, mobile]",5,7,bogo,9b98b8c7a33c4b65b9aebfe6a799e6d9
4,5,"[web, email]",20,10,discount,0b1e1539f2cc45b7b9fa7c272da2e1d7
5,3,"[web, email, mobile, social]",7,7,discount,2298d6c36e964ae4a3e7e9706d1fb8c2
6,2,"[web, email, mobile, social]",10,10,discount,fafdcd668e3743c1bb461111dcafc2a4
7,0,"[email, mobile, social]",0,3,informational,5a8bc65990b245e5a138643cd4eb9837
8,5,"[web, email, mobile, social]",5,5,bogo,f19421c1d4aa40978ebb69ca19b0e20d
9,2,"[web, email, mobile]",10,7,discount,2906b810c7d4411798c6938adc9daaa5


In [208]:
portfolio.isnull().sum()

reward        0
channels      0
difficulty    0
duration      0
offer_type    0
id            0
dtype: int64

In [209]:
#Edit portfolio dataset
#https://stackoverflow.com/questions/18889588/create-dummies-from-column-with-multiple-values-in-pandas
channels = portfolio['channels'].str.join(sep='*').str.get_dummies(sep='*')
offer_type = pd.get_dummies(portfolio["offer_type"])
offer_type

portfolio_new = pd.concat([portfolio, channels, offer_type], axis=1, sort=False)
portfolio_new.drop(['channels', 'offer_type'],axis=1)
col = portfolio_new.pop('id')
portfolio_new.insert(0, 'id', col)
portfolio_new

,id,reward,channels,difficulty,duration,offer_type,email,mobile,social,web,bogo,discount,informational
0,ae264e3637204a6fb9bb56bc8210ddfd,10,"[email, mobile, social]",10,7,bogo,1,1,1,0,1,0,0
1,4d5c57ea9a6940dd891ad53e9dbe8da0,10,"[web, email, mobile, social]",10,5,bogo,1,1,1,1,1,0,0
2,3f207df678b143eea3cee63160fa8bed,0,"[web, email, mobile]",0,4,informational,1,1,0,1,0,0,1
3,9b98b8c7a33c4b65b9aebfe6a799e6d9,5,"[web, email, mobile]",5,7,bogo,1,1,0,1,1,0,0
4,0b1e1539f2cc45b7b9fa7c272da2e1d7,5,"[web, email]",20,10,discount,1,0,0,1,0,1,0
5,2298d6c36e964ae4a3e7e9706d1fb8c2,3,"[web, email, mobile, social]",7,7,discount,1,1,1,1,0,1,0
6,fafdcd668e3743c1bb461111dcafc2a4,2,"[web, email, mobile, social]",10,10,discount,1,1,1,1,0,1,0
7,5a8bc65990b245e5a138643cd4eb9837,0,"[email, mobile, social]",0,3,informational,1,1,1,0,0,0,1
8,f19421c1d4aa40978ebb69ca19b0e20d,5,"[web, email, mobile, social]",5,5,bogo,1,1,1,1,1,0,0
9,2906b810c7d4411798c6938adc9daaa5,2,"[web, email, mobile]",10,7,discount,1,1,0,1,0,1,0


In [210]:
#Edit profile dataset
profile['age'].unique()
profile[profile['age']==118]

#Remove rows where age is 118 since it is unrealistic and missing gender values

profile_new = profile.drop(profile[profile['age']==118].index)
profile_new.isnull().sum()
#profile_new

gender              0
age                 0
id                  0
became_member_on    0
income              0
dtype: int64

In [211]:
np.unique(transcript.event)

array(['offer completed', 'offer received', 'offer viewed', 'transaction'],
      dtype=object)

In [212]:
#Edit transcript dataset

#transcript = transcript.rename(columns={'person':'id'})
#Convert event column into dummies
event_dummies = pd.get_dummies(transcript.event)



In [213]:

#Some values in the value column are offer id, offer_id and amounts. Need to separate them out

offer_list = []
amount = []
for i in range(len(transcript.value)):
    offer_list.append(list(transcript.value[i].keys())[0])
    amount.append(list(transcript.value[i].values())[0])
    
#Initialize columns
transcript['amount_cat']= amount
transcript['offer_id_cat']= offer_list

transcript['offer_id_cat'].replace({'offer id':'offer_id'}, inplace=True)
    

In [214]:
transcript

,person,event,value,time,amount_cat,offer_id_cat
0,78afa995795e4d85b5d9ceeca43f5fef,offer received,{'offer id': '9b98b8c7a33c4b65b9aebfe6a799e6d9'},0,9b98b8c7a33c4b65b9aebfe6a799e6d9,offer_id
1,a03223e636434f42ac4c3df47e8bac43,offer received,{'offer id': '0b1e1539f2cc45b7b9fa7c272da2e1d7'},0,0b1e1539f2cc45b7b9fa7c272da2e1d7,offer_id
2,e2127556f4f64592b11af22de27a7932,offer received,{'offer id': '2906b810c7d4411798c6938adc9daaa5'},0,2906b810c7d4411798c6938adc9daaa5,offer_id
3,8ec6ce2a7e7949b1bf142def7d0e0586,offer received,{'offer id': 'fafdcd668e3743c1bb461111dcafc2a4'},0,fafdcd668e3743c1bb461111dcafc2a4,offer_id
4,68617ca6246f4fbc85e91a2a49552598,offer received,{'offer id': '4d5c57ea9a6940dd891ad53e9dbe8da0'},0,4d5c57ea9a6940dd891ad53e9dbe8da0,offer_id
...,...,...,...,...,...,...
306529,b3a1272bc9904337b331bf348c3e8c17,transaction,{'amount': 1.5899999999999999},714,1.59,amount
306530,68213b08d99a4ae1b0dcb72aebd9aa35,transaction,{'amount': 9.53},714,9.53,amount
306531,a00058cf10334a308c68e7631c529907,transaction,{'amount': 3.61},714,3.61,amount
306532,76ddbd6576844afe811f1a3c0fbb5bec,transaction,{'amount': 3.5300000000000002},714,3.53,amount


In [215]:
offer_id = []
amount_clean = []

for i in range(len(transcript.value)):
    if transcript.offer_id_cat[i] == "offer_id":
         offer_id.append(transcript.amount_cat[i])
    else: 
         offer_id.append('None')

for i in range(len(transcript.value)):
    if transcript.offer_id_cat[i] == "offer_id":
         amount_clean.append(0)
    else: 
         amount_clean.append(transcript.amount_cat[i]) 
            
transcript['amount'] = amount_clean       
transcript['offer_id'] = offer_id


In [216]:
transcript_new = pd.concat([transcript,event_dummies],axis=1).drop(columns = {"event","value","amount_cat","offer_id_cat"})
transcript_new

,person,time,amount,offer_id,offer completed,offer received,offer viewed,transaction
0,78afa995795e4d85b5d9ceeca43f5fef,0,0.00,9b98b8c7a33c4b65b9aebfe6a799e6d9,0,1,0,0
1,a03223e636434f42ac4c3df47e8bac43,0,0.00,0b1e1539f2cc45b7b9fa7c272da2e1d7,0,1,0,0
2,e2127556f4f64592b11af22de27a7932,0,0.00,2906b810c7d4411798c6938adc9daaa5,0,1,0,0
3,8ec6ce2a7e7949b1bf142def7d0e0586,0,0.00,fafdcd668e3743c1bb461111dcafc2a4,0,1,0,0
4,68617ca6246f4fbc85e91a2a49552598,0,0.00,4d5c57ea9a6940dd891ad53e9dbe8da0,0,1,0,0
...,...,...,...,...,...,...,...,...
306529,b3a1272bc9904337b331bf348c3e8c17,714,1.59,None,0,0,0,1
306530,68213b08d99a4ae1b0dcb72aebd9aa35,714,9.53,None,0,0,0,1
306531,a00058cf10334a308c68e7631c529907,714,3.61,None,0,0,0,1
306532,76ddbd6576844afe811f1a3c0fbb5bec,714,3.53,None,0,0,0,1


In [229]:
#Combine all dataframes together

transcript_portfolio_new = transcript_new.merge(portfolio_new, how='left', left_on='offer_id', right_on='id').drop(columns={'id',"channels"})
transcript_portfolio_new

,person,time,amount,offer_id,offer completed,offer received,offer viewed,transaction,reward,difficulty,duration,offer_type,email,mobile,social,web,bogo,discount,informational
0,78afa995795e4d85b5d9ceeca43f5fef,0,0.00,9b98b8c7a33c4b65b9aebfe6a799e6d9,0,1,0,0,5.0,5.0,7.0,bogo,1.0,1.0,0.0,1.0,1.0,0.0,0.0
1,a03223e636434f42ac4c3df47e8bac43,0,0.00,0b1e1539f2cc45b7b9fa7c272da2e1d7,0,1,0,0,5.0,20.0,10.0,discount,1.0,0.0,0.0,1.0,0.0,1.0,0.0
2,e2127556f4f64592b11af22de27a7932,0,0.00,2906b810c7d4411798c6938adc9daaa5,0,1,0,0,2.0,10.0,7.0,discount,1.0,1.0,0.0,1.0,0.0,1.0,0.0
3,8ec6ce2a7e7949b1bf142def7d0e0586,0,0.00,fafdcd668e3743c1bb461111dcafc2a4,0,1,0,0,2.0,10.0,10.0,discount,1.0,1.0,1.0,1.0,0.0,1.0,0.0
4,68617ca6246f4fbc85e91a2a49552598,0,0.00,4d5c57ea9a6940dd891ad53e9dbe8da0,0,1,0,0,10.0,10.0,5.0,bogo,1.0,1.0,1.0,1.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
306529,b3a1272bc9904337b331bf348c3e8c17,714,1.59,None,0,0,0,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
306530,68213b08d99a4ae1b0dcb72aebd9aa35,714,9.53,None,0,0,0,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
306531,a00058cf10334a308c68e7631c529907,714,3.61,None,0,0,0,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
306532,76ddbd6576844afe811f1a3c0fbb5bec,714,3.53,None,0,0,0,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [230]:
#Merge final dataframe so we get transaction on all profiles 

df = transcript_portfolio_new.merge(profile_new,left_on='person', right_on='id').drop(columns='id')
df

,person,time,amount,offer_id,offer completed,offer received,offer viewed,transaction,reward,difficulty,...,mobile,social,web,bogo,discount,informational,gender,age,became_member_on,income
0,78afa995795e4d85b5d9ceeca43f5fef,0,0.00,9b98b8c7a33c4b65b9aebfe6a799e6d9,0,1,0,0,5.0,5.0,...,1.0,0.0,1.0,1.0,0.0,0.0,F,75,20170509,100000.0
1,78afa995795e4d85b5d9ceeca43f5fef,6,0.00,9b98b8c7a33c4b65b9aebfe6a799e6d9,0,0,1,0,5.0,5.0,...,1.0,0.0,1.0,1.0,0.0,0.0,F,75,20170509,100000.0
2,78afa995795e4d85b5d9ceeca43f5fef,132,19.89,None,0,0,0,1,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,F,75,20170509,100000.0
3,78afa995795e4d85b5d9ceeca43f5fef,132,0.00,9b98b8c7a33c4b65b9aebfe6a799e6d9,1,0,0,0,5.0,5.0,...,1.0,0.0,1.0,1.0,0.0,0.0,F,75,20170509,100000.0
4,78afa995795e4d85b5d9ceeca43f5fef,144,17.78,None,0,0,0,1,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,F,75,20170509,100000.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
272757,9fcbff4f8d7241faa4ab8a9d19c8a812,504,0.00,3f207df678b143eea3cee63160fa8bed,0,0,1,0,0.0,0.0,...,1.0,0.0,1.0,0.0,0.0,1.0,M,47,20171013,94000.0
272758,9fcbff4f8d7241faa4ab8a9d19c8a812,576,0.00,4d5c57ea9a6940dd891ad53e9dbe8da0,0,1,0,0,10.0,10.0,...,1.0,1.0,1.0,1.0,0.0,0.0,M,47,20171013,94000.0
272759,9fcbff4f8d7241faa4ab8a9d19c8a812,576,0.00,4d5c57ea9a6940dd891ad53e9dbe8da0,0,0,1,0,10.0,10.0,...,1.0,1.0,1.0,1.0,0.0,0.0,M,47,20171013,94000.0
272760,3045af4e98794a04a5542d3eac939b1f,576,0.00,4d5c57ea9a6940dd891ad53e9dbe8da0,0,1,0,0,10.0,10.0,...,1.0,1.0,1.0,1.0,0.0,0.0,F,58,20161020,78000.0
